In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install Sastrawi gensim scikit-learn joblib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 70.3 MB/s eta 0:00:00


In [5]:
import pandas as pd
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [ ]:
# Load Dataset
df = pd.read_csv('/content/drive/MyDrive/deteksi_cd-indo/dataset.csv')
df.head()

,id,text,label
0,1,ketika saya sudah berusaha semaksimal mungkin ...,All-or-nothing
1,2,Saya ketika SD hingga SMP dulu beranggapan bah...,All-or-nothing
2,3,Saya berfikir jika tidak lolos MBKM maka masa ...,All-or-nothing
3,4,"Di desa saya dulu ada dua kelompok pertemanan,...",All-or-nothing
4,5,Saya hanya berteman dengan orang yang tidak me...,All-or-nothing


In [7]:
print(df['label'].value_counts())

label
No Distortion                    3179
Jumping to Conclusions            827
Labeling                          555
Overgeneralization                454
Personalization and Blame         437
Magnification or Minimization     395
Should statement                  371
Emotional Reasoning               334
Mental filter                     322
All-or-nothing                    300
Discounting the positives         241
Should statements                 107
Name: count, dtype: int64


In [8]:
df['label'] = df['label'].replace({'Should statements':'Should statement'})

In [9]:
print(df['label'].value_counts())

label
No Distortion                    3179
Jumping to Conclusions            827
Labeling                          555
Should statement                  478
Overgeneralization                454
Personalization and Blame         437
Magnification or Minimization     395
Emotional Reasoning               334
Mental filter                     322
All-or-nothing                    300
Discounting the positives         241
Name: count, dtype: int64


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7522 entries, 0 to 7521
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      7522 non-null   int64 
 1   text    7522 non-null   object
 2   label   7522 non-null   object
dtypes: int64(1), object(2)
memory usage: 176.4+ KB


In [11]:
df['label'].value_counts()

,count
label,
No Distortion,3179
Jumping to Conclusions,827
Labeling,555
Should statement,478
Overgeneralization,454
Personalization and Blame,437
Magnification or Minimization,395
Emotional Reasoning,334
Mental filter,322


In [12]:
# Hapus duplikasi dan baris kosong
df = df.drop_duplicates(subset=['text']).reset_index(drop=True)
df = df[df['text'].notnull() & (df['text'].str.strip() != '')].reset_index(drop=True)
len(df)

7484

In [13]:
# cleaning modular
emoji_pattern = re.compile("[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF"
                           "\U0001F680-\U0001F6FF\U0001F1E0-\U0001F1FF]+", flags=re.UNICODE)

def clean_basic(s):
    if not isinstance(s,str): return ""
    s = s.strip().lower()
    s = re.sub(r'http\S+|www\.\S+','', s)
    s = re.sub(r'@\w+|#\w+','', s)
    s = emoji_pattern.sub('', s)
    s = re.sub(r'(.)\1{2,}', r'\1\1', s)
    s = re.sub(r'[^0-9a-z\s]', ' ', s)   # hapus simbol kecuali angka/letter
    s = re.sub(r'\s+', ' ', s).strip()
    return s
df['text_clean'] = df['text'].apply(clean_basic)
df[['text', 'text_clean']].head(10)

,text,text_clean
0,ketika saya sudah berusaha semaksimal mungkin ...,ketika saya sudah berusaha semaksimal mungkin ...
1,Saya ketika SD hingga SMP dulu beranggapan bah...,saya ketika sd hingga smp dulu beranggapan bah...
2,Saya berfikir jika tidak lolos MBKM maka masa ...,saya berfikir jika tidak lolos mbkm maka masa ...
3,"Di desa saya dulu ada dua kelompok pertemanan,...",di desa saya dulu ada dua kelompok pertemanan ...
4,Saya hanya berteman dengan orang yang tidak me...,saya hanya berteman dengan orang yang tidak me...
5,Ketika aku mengikuti lomba dimana aku menghara...,ketika aku mengikuti lomba dimana aku menghara...
6,"pernah waktu SD-SMP, saya merasa kalo saya dap...",pernah waktu sd smp saya merasa kalo saya dape...
7,Saya pernah tidak mendapatkan juara dalam sebu...,saya pernah tidak mendapatkan juara dalam sebu...
8,"Ketika berpacaran, saya hanya berpikiran bahwa...",ketika berpacaran saya hanya berpikiran bahwa ...
9,Saya pernah mengikuti suatu kompetisi untuk me...,saya pernah mengikuti suatu kompetisi untuk me...


In [14]:
slang = {'gak':'tidak','ga':'tidak','tdk':'tidak','yg':'yang','dgn':'dengan','sndiri':'sendiri'}
def normalize_slang(s):
    words = s.split()
    return ' '.join([slang.get(w,w) for w in words])
df['text_clean'] = df['text_clean'].apply(normalize_slang)
df[['text', 'text_clean']].head(10)

,text,text_clean
0,ketika saya sudah berusaha semaksimal mungkin ...,ketika saya sudah berusaha semaksimal mungkin ...
1,Saya ketika SD hingga SMP dulu beranggapan bah...,saya ketika sd hingga smp dulu beranggapan bah...
2,Saya berfikir jika tidak lolos MBKM maka masa ...,saya berfikir jika tidak lolos mbkm maka masa ...
3,"Di desa saya dulu ada dua kelompok pertemanan,...",di desa saya dulu ada dua kelompok pertemanan ...
4,Saya hanya berteman dengan orang yang tidak me...,saya hanya berteman dengan orang yang tidak me...
5,Ketika aku mengikuti lomba dimana aku menghara...,ketika aku mengikuti lomba dimana aku menghara...
6,"pernah waktu SD-SMP, saya merasa kalo saya dap...",pernah waktu sd smp saya merasa kalo saya dape...
7,Saya pernah tidak mendapatkan juara dalam sebu...,saya pernah tidak mendapatkan juara dalam sebu...
8,"Ketika berpacaran, saya hanya berpikiran bahwa...",ketika berpacaran saya hanya berpikiran bahwa ...
9,Saya pernah mengikuti suatu kompetisi untuk me...,saya pernah mengikuti suatu kompetisi untuk me...


In [15]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stem_text(s):
    return stemmer.stem(s)

df['text_stem'] = df['text_clean'].apply(stem_text)
df[['text_clean', 'text_stem']].head(10)

,text_clean,text_stem
0,ketika saya sudah berusaha semaksimal mungkin ...,ketika saya sudah usaha maksimal mungkin tapi ...
1,saya ketika sd hingga smp dulu beranggapan bah...,saya ketika sd hingga smp dulu anggap bahwa ke...
2,saya berfikir jika tidak lolos mbkm maka masa ...,saya berfikir jika tidak lolos mbkm maka masa ...
3,di desa saya dulu ada dua kelompok pertemanan ...,di desa saya dulu ada dua kelompok teman nah s...
4,saya hanya berteman dengan orang yang tidak me...,saya hanya teman dengan orang yang tidak sakit...
5,ketika aku mengikuti lomba dimana aku menghara...,ketika aku ikut lomba mana aku harap juara 1 a...
6,pernah waktu sd smp saya merasa kalo saya dape...,pernah waktu sd smp saya rasa kalo saya dapet ...
7,saya pernah tidak mendapatkan juara dalam sebu...,saya pernah tidak dapat juara dalam buah lomba...
8,ketika berpacaran saya hanya berpikiran bahwa ...,ketika pacar saya hanya pikir bahwa lama masih...
9,saya pernah mengikuti suatu kompetisi untuk me...,saya pernah ikut suatu kompetisi untuk rebut j...


In [20]:
df['tokens'] = df['text_stem'].apply(lambda x: x.split() if isinstance(x, str) else x)
# Hapus baris dengan token kosong
df = df[df['tokens'].map(len) > 0].reset_index(drop=True)
df.head(10)

,id,text,label,text_clean,text_stem,tokens
0,1,ketika saya sudah berusaha semaksimal mungkin ...,All-or-nothing,ketika saya sudah berusaha semaksimal mungkin ...,ketika saya sudah usaha maksimal mungkin tapi ...,"[ketika, saya, sudah, usaha, maksimal, mungkin..."
1,2,Saya ketika SD hingga SMP dulu beranggapan bah...,All-or-nothing,saya ketika sd hingga smp dulu beranggapan bah...,saya ketika sd hingga smp dulu anggap bahwa ke...,"[saya, ketika, sd, hingga, smp, dulu, anggap, ..."
2,3,Saya berfikir jika tidak lolos MBKM maka masa ...,All-or-nothing,saya berfikir jika tidak lolos mbkm maka masa ...,saya berfikir jika tidak lolos mbkm maka masa ...,"[saya, berfikir, jika, tidak, lolos, mbkm, mak..."
3,4,"Di desa saya dulu ada dua kelompok pertemanan,...",All-or-nothing,di desa saya dulu ada dua kelompok pertemanan ...,di desa saya dulu ada dua kelompok teman nah s...,"[di, desa, saya, dulu, ada, dua, kelompok, tem..."
4,5,Saya hanya berteman dengan orang yang tidak me...,All-or-nothing,saya hanya berteman dengan orang yang tidak me...,saya hanya teman dengan orang yang tidak sakit...,"[saya, hanya, teman, dengan, orang, yang, tida..."
5,6,Ketika aku mengikuti lomba dimana aku menghara...,All-or-nothing,ketika aku mengikuti lomba dimana aku menghara...,ketika aku ikut lomba mana aku harap juara 1 a...,"[ketika, aku, ikut, lomba, mana, aku, harap, j..."
6,7,"pernah waktu SD-SMP, saya merasa kalo saya dap...",All-or-nothing,pernah waktu sd smp saya merasa kalo saya dape...,pernah waktu sd smp saya rasa kalo saya dapet ...,"[pernah, waktu, sd, smp, saya, rasa, kalo, say..."
7,8,Saya pernah tidak mendapatkan juara dalam sebu...,All-or-nothing,saya pernah tidak mendapatkan juara dalam sebu...,saya pernah tidak dapat juara dalam buah lomba...,"[saya, pernah, tidak, dapat, juara, dalam, bua..."
8,9,"Ketika berpacaran, saya hanya berpikiran bahwa...",All-or-nothing,ketika berpacaran saya hanya berpikiran bahwa ...,ketika pacar saya hanya pikir bahwa lama masih...,"[ketika, pacar, saya, hanya, pikir, bahwa, lam..."
9,10,Saya pernah mengikuti suatu kompetisi untuk me...,All-or-nothing,saya pernah mengikuti suatu kompetisi untuk me...,saya pernah ikut suatu kompetisi untuk rebut j...,"[saya, pernah, ikut, suatu, kompetisi, untuk, ..."


In [21]:
# Bersihkan noise jika ada sample pendek
df = df[df['tokens'].map(len) >= 3].reset_index(drop=True)
len(df)

7484

In [22]:
# Encoding label
label2id = {lab:i for i,lab in enumerate(sorted(df['label'].unique()))}
df['label_id'] = df['label'].map(label2id)
df.head()

,id,text,label,text_clean,text_stem,tokens,label_id
0,1,ketika saya sudah berusaha semaksimal mungkin ...,All-or-nothing,ketika saya sudah berusaha semaksimal mungkin ...,ketika saya sudah usaha maksimal mungkin tapi ...,"[ketika, saya, sudah, usaha, maksimal, mungkin...",0
1,2,Saya ketika SD hingga SMP dulu beranggapan bah...,All-or-nothing,saya ketika sd hingga smp dulu beranggapan bah...,saya ketika sd hingga smp dulu anggap bahwa ke...,"[saya, ketika, sd, hingga, smp, dulu, anggap, ...",0
2,3,Saya berfikir jika tidak lolos MBKM maka masa ...,All-or-nothing,saya berfikir jika tidak lolos mbkm maka masa ...,saya berfikir jika tidak lolos mbkm maka masa ...,"[saya, berfikir, jika, tidak, lolos, mbkm, mak...",0
3,4,"Di desa saya dulu ada dua kelompok pertemanan,...",All-or-nothing,di desa saya dulu ada dua kelompok pertemanan ...,di desa saya dulu ada dua kelompok teman nah s...,"[di, desa, saya, dulu, ada, dua, kelompok, tem...",0
4,5,Saya hanya berteman dengan orang yang tidak me...,All-or-nothing,saya hanya berteman dengan orang yang tidak me...,saya hanya teman dengan orang yang tidak sakit...,"[saya, hanya, teman, dengan, orang, yang, tida...",0


In [23]:
print(df['label'].value_counts())

label
No Distortion                    3155
Jumping to Conclusions            827
Labeling                          555
Should statement                  477
Overgeneralization                452
Personalization and Blame         436
Magnification or Minimization     387
Emotional Reasoning               332
Mental filter                     322
All-or-nothing                    300
Discounting the positives         241
Name: count, dtype: int64


In [ ]:
# Simpan hasil preprocessing
Path = '/content/drive/MyDrive/deteksi_cd-indo/preprocessed.csv'
df.to_csv(Path, index=False)